In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
%matplotlib inline

In [2]:
with h5py.File('train_catvnoncat.h5', 'r') as file:
    train_set_x_original = np.array(file['train_set_x'][:])
    train_set_y = np.array(file['train_set_y'][:])
    train_set_y = train_set_y.reshape((1, train_set_y.shape[0]))
    
with h5py.File('test_catvnoncat.h5', 'r') as file:
    classes = np.array(file['list_classes'][:])
    test_set_x_original = np.array(file['test_set_x'][:])
    test_set_y = np.array(file['test_set_y'][:])
    test_set_y = test_set_y.reshape((1, test_set_y.shape[0]))

In [3]:
m_train = train_set_x_original.shape[0]
m_test = test_set_x_original.shape[0]
num_px = train_set_x_original.shape[1]

In [4]:
train_set_x_flatten = train_set_x_original.reshape(train_set_x_original.shape[0], -1).T

In [5]:
test_set_x_flatten = test_set_x_original.reshape(test_set_x_original.shape[0], -1).T

In [6]:
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

In [11]:
# sigmoid function is defined as sigmoid(z) = 1/(1 + np.exp(z*(-1)))
def sigmoid(z):
    # z is an np array
    a = 1 / (1 + np.exp(z*(-1)))
    return a

In [13]:
def initialize_with_zeros(dim):
    w = np.zeros(shape=(dim, 1))
    b = 0
    return w, b

In [26]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    J = np.sum(np.dot(Y, np.log(A).T) + np.dot((1-Y), np.log((1-A)).T))
    J = J/m * -1
    dZ = A - Y
    dw = np.dot(X, dZ.T)
    dw = dw/m
    db = np.sum(dZ)
    db = db/m
    J = np.squeeze(J)
    grads = {"dw": dw,
             "db": db}

    return grads, J

In [27]:
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99993216]
 [1.99980262]]
db = 0.49993523062470574
cost = 6.000064773192205
